给你一个 n * n 矩阵 grid ，矩阵由若干 0 和 1 组成。请你用四叉树表示该矩阵 grid 。

你需要返回能表示矩阵 grid 的 四叉树 的根结点。

四叉树数据结构中，每个内部节点只有四个子节点。此外，每个节点都有两个属性：
```
val：储存叶子结点所代表的区域的值。1 对应 True，0 对应 False。注意，当 isLeaf 为 False 时，你可以把 True 或者 False 赋值给节点，两种值都会被判题机制 接受 。
isLeaf: 当这个节点是一个叶子结点时为 True，如果它有 4 个子节点则为 False 。
class Node {
    public boolean val;
    public boolean isLeaf;
    public Node topLeft;
    public Node topRight;
    public Node bottomLeft;
    public Node bottomRight;
}
```
我们可以按以下步骤为二维区域构建四叉树：

如果当前网格的值相同（即，全为 0 或者全为 1），将 isLeaf 设为 True ，将 val 设为网格相应的值，并将四个子节点都设为 Null 然后停止。
如果当前网格的值不同，将 isLeaf 设为 False， 将 val 设为任意值，然后如下图所示，将当前网格划分为四个子网格。
使用适当的子网格递归每个子节点。

In [23]:
class Node:
    def __init__(self, val, isLeaf, topLeft, topRight, bottomLeft, bottomRight):
        self.val = val
        self.isLeaf = isLeaf
        self.topLeft = topLeft
        self.topRight = topRight
        self.bottomLeft = bottomLeft
        self.bottomRight = bottomRight


list_list = [[1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1],
             [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0]]


#  利用递归
class Solution:
    @staticmethod
    def construct(grid: list[list[int]]) -> 'Node':
        def create_node(row_start: int, column_start: int, row_end: int, column_end: int) -> 'Node':
            # 判断该区域值是否相同 相同则创建叶子节点，否则创建根节点
            if all(grid[row_start][column_start] == grid[r][c] for r in range(row_start, row_end) for c in
                   range(column_start, column_end)):
                return Node(grid[row_start][column_start] == 1, True, None, None, None, None)
            else:
                return Node(
                    True,
                    False,
                    create_node(row_start, column_start, (row_start + row_end) // 2, (column_start + column_end) // 2),
                    create_node(row_start, (column_start + column_end) // 2, (row_start + row_end) // 2, column_end),
                    create_node((row_start + row_end) // 2, column_start, row_end, (column_start + column_end) // 2),
                    create_node((row_start + row_end) // 2, (column_start + column_end) // 2, row_end, column_end)
                )

        return create_node(0, 0, len(grid), len(grid[0]))


sol = Solution()
sol.construct(list_list)


可以利用 前缀和矩阵 优化对同一区域值的判断
```python
class Solution:
    def construct(self, grid: List[List[int]]) -> 'Node':
        n = len(grid)
        # pre 前缀和的矩阵会比原矩阵大一些，原矩阵的坐标对应的是元素，前缀和的矩阵坐标对应的是交点
        pre = [[0] * (n + 1) for _ in range(n + 1)]
        for i in range(1, n + 1):
            for j in range(1, n + 1):
                pre[i][j] = pre[i - 1][j] + pre[i][j - 1] - pre[i - 1][j - 1] + grid[i - 1][j - 1]
        print(pre)
        def getSum(r0: int, c0: int, r1: int, c1: int) -> int:
            return pre[r1][c1] - pre[r1][c0] - pre[r0][c1] + pre[r0][c0]

        def dfs(r0: int, c0: int, r1: int, c1: int) -> 'Node':
            total = getSum(r0, c0, r1, c1)
            if total == 0:
                return Node(False, True)
            if total == (r1 - r0) * (c1 - c0):
                return Node(True, True)
            return Node(
                True,
                False,
                dfs(r0, c0, (r0 + r1) // 2, (c0 + c1) // 2),
                dfs(r0, (c0 + c1) // 2, (r0 + r1) // 2, c1),
                dfs((r0 + r1) // 2, c0, r1, (c0 + c1) // 2),
                dfs((r0 + r1) // 2, (c0 + c1) // 2, r1, c1),
            )

        return dfs(0, 0, n, n)
```

要构建一个四叉树而不使用递归，可以使用栈或队列来管理需要处理的节点。下面是使用栈来实现这一方法的解决方案：

```python
class Node:
    def __init__(self, val, isLeaf, topLeft, topRight, bottomLeft, bottomRight):
        self.val = val
        self.isLeaf = isLeaf
        self.topLeft = topLeft
        self.topRight = topRight
        self.bottomLeft = bottomLeft
        self.bottomRight = bottomRight

grid = [[1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0]]

class Solution:
    def construct(self, grid: list[list[int]]) -> 'Node':
        n = len(grid)
        stack = [(0, 0, n, n)]
        node_stack = []
        
        while stack:
            r0, c0, r1, c1 = stack.pop()
            if all(grid[i][j] == grid[r0][c0] for i in range(r0, r1) for j in range(c0, c1)):
                node_stack.append(Node(grid[r0][c0] == 1, True, None, None, None, None))
            else:
                node = Node(True, False, None, None, None, None)
                node_stack.append(node)
                mid_row, mid_col = (r0 + r1) // 2, (c0 + c1) // 2
                
                stack.append((mid_row, mid_col, r1, c1))  # bottomRight
                stack.append((mid_row, c0, r1, mid_col))  # bottomLeft
                stack.append((r0, mid_col, mid_row, c1))  # topRight
                stack.append((r0, c0, mid_row, mid_col))  # topLeft

        def fill_nodes():
            index = len(node_stack) - 1
            while index >= 0:
                node = node_stack[index]
                if not node.isLeaf:
                    node.bottomRight = node_stack.pop()
                    node.bottomLeft = node_stack.pop()
                    node.topRight = node_stack.pop()
                    node.topLeft = node_stack.pop()
                index -= 1
            return node_stack.pop()
        
        return fill_nodes()

sol = Solution()
four_tree = sol.construct(grid)
```

在这个代码中：
1. `construct` 函数初始化一个栈，用于存储需要处理的网格区域。
2. 它通过检查每个区域是否可以表示为一个叶节点来处理每个区域。如果可以，它会将一个叶节点添加到 `node_stack`。
3. 如果一个区域不能表示为叶节点，它会将该区域分成四个子区域，将这些子区域推入栈中，并创建一个非叶节点。
4. 一旦所有区域都处理完毕，`fill_nodes` 函数通过从 `node_stack` 中弹出节点并将子节点连接到它们各自的父节点，重新组装节点的层次结构。
5. 该函数返回构建的四叉树的根节点。

这种迭代方法使用栈来避免递归，适用于递归深度有限的环境。